In [1]:
import pandas as pd, numpy as np

In [89]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')

In [90]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [91]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]

In [92]:
dc=dc.set_index('uid')
dm=dm.set_index('uid')

In [94]:
dmi=dm.loc[dc.index.dropna()]

In [95]:
df=dmi.merge(dc).drop_duplicates()

Working df ready. Now infer location.

In [97]:
locs=df[['category','icategory','tags','title','content']]

In [98]:
locs

,category,icategory,tags,title,content
0,NaN,NaN,None,Súlyosbodó medvegondok Zsögödfürdőn,Nem találnak megoldást a hatóságok a Csíkszere...
1,NaN,NaN,None,Idén nem okoztak gondot a brassói medvék,Csupán négy medvét láttak a hibernálási idősza...
2,NaN,NaN,None,Medve harapott meg egy fiút Brassóban,Medve támadt két fiúra a brassói új negyed árv...
3,NaN,kronika,None,„Háziasult” medveállomány,Az emberi tevékenység átalakította a vadállato...
4,NaN,kronika,None,Négyszemközt a medvével,Romániában él az egyetlen európai életképes me...
5,NaN,NaN,None,Medvéktől rettegnek Háromszéken,"Medvék tartják rettegésben a lemhényi, sepsima..."
6,NaN,NaN,None,Lelőtték a támadó zernyesti medvét,Brassó megye – Lelőtték vasárnap hajnalban azt...
7,NaN,NaN,None,Vadorzás miatt vizsgálódnak,Hargita megye: Vadorzás miatt indított kivizsg...
8,NaN,NaN,None,Medveveszély Háromszéken,Medvék tartják rettegésben a háromszéki falvak...
9,NaN,NaN,None,Csökkentenék a medveállományt,Szabályozni szükséges a medveállományt Hargita...


In [103]:
import json, requests

In [104]:
r=requests.get('http://szekelydata.csaladen.es/lakossag/dict.json')

In [112]:
lak=json.loads(r.content)

In [114]:
lak={lak[i][3]:lak[i][2] for i in lak}

In [117]:
#osszes erdelyi magyar telepulesnev
laks=lak.keys()

In [140]:
locs.loc[i]['title'].split()

['Kételkednek', 'a', 'tervezett', 'medveexport', 'sikerében']

In [146]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [167]:
#look for perfect matches
decoded_locs={}
for i in locs.index:
    if '[' in locs.loc[i]['tags']:
        k=locs.loc[i]['tags'].replace("'",'"')
        for j in json.loads(k):
            if j in laks:
                if i not in decoded_locs:decoded_locs[i]=[]
                decoded_locs[i].append(j)
                print('1',k,j)
    for j in locs.loc[i]['title'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('2',j)
    for j in locs.loc[i]['content'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('3',j)
    for j in locs.loc[i]['title'].split():
        for k in laks:
            if levenshteinDistance(j,k)<3:
                if i not in decoded_locs:decoded_locs[i]=[]
                decoded_locs[i].append(k)
                print('4',j,k)
    for j in locs.loc[i]['content'].split():
        for k in laks:
            if levenshteinDistance(j,k)<3:
                if i not in decoded_locs:decoded_locs[i]=[]
                decoded_locs[i].append(k)
                print('5',j,k)

5 négy Réty
5 két Réty
5 távon Zágon
5 fontos Cintos
3 Uzon
5 Uzon Uzon
3 Homoródszentmárton
5 Homoródszentmárton Homoródszentmárton
3 Bodzaforduló
5 Bodzaforduló Bodzaforduló
5 Málnáson Málnás
5 Bodzafordulón Bodzaforduló
3 Bereck
5 Bereck Bereck
5 Bereckbe Bereck
5 ezen Uzon
3 Kovászna
5 halt Bala
5 Kovászna Kovászna
5 Komandó Kommandó
3 Kovászna
4 kovásznai Kovászna
5 Kovászna Kovászna
3 Kovászna
5 Kovászna Kovászna
4 két Réty
5 két Réty
5 két Réty
5 marosvásárhelyi Marosvásárhely
3 Marosvásárhely
5 Marosvásárhely Marosvásárhely
5 Magyar Magyaró
5 Négy Réty
5 két, Réty
5 gyergyószentmiklósi Gyergyószentmiklós
3 Kovászna
5 Kovászna Kovászna
5 Bikfalva Mikefalva
5 Bikfalva Makfalva
5 falu Bala
5 falu Bala
5 uzoni Uzon
5 számos Ádámos
3 Gyergyószentmiklós
4 Gyergyószentmiklóson Gyergyószentmiklós
5 Gyergyószentmiklós Gyergyószentmiklós
5 két Réty
3 Gyergyószentmiklós
4 Gyergyószentmiklósra Gyergyószentmiklós
5 Gyergyószentmiklós Gyergyószentmiklós
5 sepsiszentgyörgyi Sepsiszentgyörgy
5

In [176]:
predicted_locs=[]
for i in range(len(locs.index)):
    if locs.index[i] in decoded_locs: 
        dummy=list(set(decoded_locs[locs.index[i]]))
        if len(dummy)==1:
            dummy=dummy[0]
        else:
            dummy=repr(decoded_locs[locs.index[i]])
        predicted_locs.append(dummy)
    else: predicted_locs.append('')

In [177]:
locs['predicted_locs']=predicted_locs
locs['actual_locs']=predicted_locs

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [178]:
lm=locs[['title','content','predicted_locs','actual_locs']]

In [179]:
lm.to_excel('data/szekelyhon_locs.xlsx')
lm.to_excel('data/szekelyhon_locs_curated.xlsx')

Open `szekelyhon_locs_curated` and manually verify predicted locations. Fill where necessary.